In [1]:
n_cores = -1

In [2]:
import numpy as np
import pandas as pd

In [3]:
from bisect import bisect_left
from scipy.sparse import csr_matrix, save_npz

In [4]:
data = np.load('inga_out.npz')

In [5]:
mass = data['arr_0']
mass

array([ 552.293448,  338.046024,  724.221464, ...,  328.072907,
        262.19328 ,  636.278192])

In [6]:
name = data['arr_1']
name

array(['UNPD98266', 'UNPD207163', 'UNPD3499', ..., 'UNPD98267',
       'UNPD47332', 'UNPD101003'], dtype=object)

In [7]:
massabund = data['arr_2']
massabund

array([[  41.00329 ,    1.688456],
       [  43.01894 ,    2.135631],
       [  55.01894 ,    1.105409],
       ..., 
       [ 549.23414 ,   24.377134],
       [ 551.24979 ,   22.666363],
       [ 591.2447  ,    5.496205]])

In [8]:
blockind = data['arr_3']
blockind

array([     0,      0,      0, ..., 220988, 220988, 220988], dtype=uint32)

### Splitting

In [9]:
def splittingSparse(massabund, blockind, threshold):
    """
    Using splitting strategy to generate sparse matrix.
    """
    values = np.sort(massabund[:,0])
    
    splits = []
    recurSplitting(values, splits, threshold)
    splits.sort()
    np.save('splits_splitting_'+str(threshold), splits)
    
    valueRange = np.array(splits)
    
    splits = []
    recurSplitting(values, splits, threshold)
    splits.sort()
    valueRange = np.array(splits)
    
    groups = np.array([bisect_left(valueRange, m) for m in massabund[:,0]])
        
    features = generateSparseMatrix(massabund, blockind, groups)
    save_npz('splitting_'+str(threshold), features)
    
    return features

In [10]:
def recurSplitting(arr, res, threshold):
    """
    A recursive method to split values into groups until 
    within each group there are no two consecutive values
    which have differences larger than the given threshold.
    """
    if threshold <= 0:
        raise ValueError('Threshold should be positive.')
        
    n = len(arr)
    maxInterval = -1
    maxIdx = -1
    for i in range(1,n):
        diff = arr[i] - arr[i-1]
        if diff > maxInterval:
            maxInterval = diff
            maxIdx = i
            
    if maxInterval < threshold:
        return
    else:
        res.append((arr[maxIdx-1]+arr[maxIdx])/2)
        arr1 = arr[0:maxIdx]
        recurSplitting(arr1, res, threshold)
        arr2 = arr[maxIdx:n]
        recurSplitting(arr2, res, threshold)

In [11]:
def generateSparseMatrix(massabund, blockind, groups):
    """
    Generating the corresponding sparse matrix based on the groups.
    """
    newmassabund = np.concatenate([massabund, blockind.reshape(-1,1), groups.reshape(-1,1)], axis=1)
    data = newmassabund[:,1]
    row = newmassabund[:,2]
    col = newmassabund[:,3]
    n_rows = int(row.max())+1
    n_cols = int(col.max())+1
    
    result = csr_matrix((data, (row, col)), shape=(n_rows, n_cols))
    
    goodCol = result.sum(axis=0) > 0
    newresult = result[:,np.ravel(goodCol)]
    
    return newresult

In [12]:
for n in [0.2, 0.1, 0.05]:
    splittingSparse(massabund, blockind, n)